In [1]:
from get_sinks import get_sinks_from_content
import pandas as pd
import numpy as np
from extract_sink_features import sink_predict_class, SINK_TYPE_DICT

In [2]:
df = pd.read_excel('new_seq.xlsx')
print(df.columns)
opcode_list = df['opcode_sequence'].tolist()
label_list = np.array(df['label'].tolist())

# sink_type_list = []
# for code in code_list:
#     sink_type_list.append(get_sinks_from_content(code)[0])
# sink_type_list = np.array(sink_type_list)

print(len(opcode_list))
print(len(label_list))

Index(['opcode_sequence', 'label'], dtype='object')
14490
14490


In [3]:
from gensim.models import Word2Vec
import nltk

corpus = []
for opcode in opcode_list:
#     corpus.append(nltk.word_tokenize(opcode))
    corpus.append(opcode.split(' '))
corpus = np.array(corpus)
w2v_model = Word2Vec(corpus, size=128, window=32, min_count=1, sg=0)

X_w2v = []
leng = []
max_seq_len = 128
for opcode_seq in opcode_list:
    opcode_seq = opcode_seq.split(' ')
    w2v_seq = []
    for opcode in opcode_seq:
        w2v_seq.append(w2v_model[opcode])
    w2v_seq = np.array(w2v_seq)
    # if(w2v_seq.shape[0] < max_seq_len):
    #     # print('small')
    #     zeros = [np.zeros(len(w2v_model['ECHO'])) for i in range(max_seq_len-w2v_seq.shape[0])]
    #     w2v_seq = np.concatenate((np.array(w2v_seq), zeros))
    X_w2v.append(w2v_seq)
    leng.append(len(w2v_seq))
print(np.array(X_w2v).shape)
print(min(leng))
print(max(leng))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(14490,)
7
108


In [4]:
from keras.preprocessing import sequence

X_w2v_std = np.array(sequence.pad_sequences(value=0.0, maxlen=max_seq_len, sequences=X_w2v))
print(X_w2v_std.shape)

Using TensorFlow backend.


(14490, 128, 128)


In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X_w2v_std, label_list, 
                                                    random_state=2019, test_size = 0.1, stratify=label_list)
# y_train_init = y_train
y_train = to_categorical(y_train, 2)
# y_test = to_categorical(y_test, 2).astype('int')
X_train_std, X_test_std = np.array(X_train), np.array(X_test)

print(X_train_std.shape)
print(y_train.shape)
print(X_test_std.shape)
print(y_test.shape)

(13041, 128, 128)
(13041, 2)
(1449, 128, 128)
(1449,)


In [6]:
from tensorflow.keras.layers import Dense, Flatten, Embedding, Dropout, Bidirectional, LSTM, Masking
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(max_seq_len, len(w2v_model['ECHO']))))
# model.add(Bidirectional(LSTM(units=128, return_state=True, dropout=0.25), merge_mode='concat'))
model.add(Bidirectional(LSTM(units=128, dropout=0.25, recurrent_dropout=0.25)))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='nadam',
             metrics=['accuracy'])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
epoch_num = 10
history1 = model.fit(X_train_std, y_train, epochs=epoch_num, validation_split=0.2)

In [9]:
from sklearn import metrics
predict_target = model.predict_classes(X_test_std)
print(metrics.classification_report(y_test, predict_target, 
                                    target_names=['Good sanitize', 'Bad sanitize']))

TypeError: Expected int32 passed to parameter 'y' of op 'NotEqual', got 0.0 of type 'float' instead. Error: Expected int32, got 0.0 of type 'float' instead.